In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

# Extract

In [2]:
df_absen = pd.read_excel("ABSEN INSENTIF JUNI 2024.xlsx", dtype={"Employee ID": "str"})[
    [
        "Company",
        "Employee ID",
        "Full Name",
        "Job Position",
        "Branch Code",
        "Branch",
        "Hari Kerja",
    ]
]

In [3]:
sales_24 = pd.concat(
    [
        pd.read_parquet(
            f"s3://mega-dev-lake/Staging/Sales/akumulasi/daily/2024/{month}/data.parquet"
        )
        for month in ["06"]
    ]
)
sales_24 = sales_24[(sales_24["channel"] != "ONLINE")]
df_sales = (
    sales_24.groupby(["dataareaid", "axcode", "month"], as_index=False)
    .agg({"cust_paid_peritem": "sum", "target_daily": "sum"})
    .rename(columns={"cust_paid_peritem": "grand_total_rk"})
)

In [4]:
sales_24_growth = pd.concat(
    [
        pd.read_parquet(
            f"s3://mega-dev-lake/Staging/Sales/akumulasi/growth/2024/{month}/data.parquet"
        )
        for month in ["06"]
    ]
)

In [5]:
df_sales = df_sales.merge(
    sales_24_growth.groupby(["dataareaid", "axcode", "month"], as_index=False).agg(
        {"cust_paid_peritem_last_year": "sum"}
    ),
    "left",
    on=["dataareaid", "axcode", "month"],
)

In [6]:
# Master
master_raw = pd.read_parquet(
    "s3://mega-dev-lake/Staging/Master/Master Store/2024/data.parquet"
)
master = master_raw[master_raw["month"].isin(["June"])]
master = master.rename(columns={"area head": "areahead", "city head": "cityhead"})

# Transformation

## Transformasi df_sales_master

In [7]:
def calculate_growth(df):
    df["growth"] = (df["sales_24"] - df["sales_23"]) / df["sales_23"]
    df["growth"] = df["growth"].replace([np.inf, -np.inf, np.nan], 0)
    return df


def calculate_ach(df):
    df["achievement"] = df["sales_24"] / df["target"]
    df["achievement"] = df["achievement"].replace([np.inf, -np.inf, np.nan], 0)
    return df


def calculate_growth_req(df):
    df["growthreq"] = np.where(df["growth"] >= 0.10, "Yes", "No")
    return df

In [8]:
df_sales = df_sales.rename(
    columns={
        "grand_total_rk": "sales_24",
        "cust_paid_peritem_last_year": "sales_23",
        "target_daily": "target",
    }
)

In [9]:
master_filtered = master[
    [
        "dataareaid",
        "axcode",
        "cscode",
        "stdname",
        "sssg",
        "areahead",
        "cityhead",
    ]
]

In [10]:
df_sales_master = master_filtered.merge(df_sales, "right", on=["dataareaid", "axcode"])

In [11]:
df_sales_master = calculate_growth(df_sales_master)
df_sales_master = calculate_ach(df_sales_master)
df_sales_master = calculate_growth_req(df_sales_master)
df_sales_master["growthreq"] = np.where(
    df_sales_master["sssg"] != "SSSG", "Yes", df_sales_master["growthreq"]
)

In [12]:
df_sales_master.head(5)

,dataareaid,axcode,cscode,stdname,sssg,areahead,cityhead,month,sales_24,target,sales_23,growth,achievement,growthreq
0,mgl,C0001,CS1618,SHOWROOM PM EMPORIUM PLUIT,SSSG,Henri Mediyanto,None,June,284075670.0,353000000.0,300719275.0,-0.055346,0.804747,No
1,mgl,C0002,CS1619,SHOWROOM PM MEGA MALL BATAM,SSSG,Rika Yuanda,None,June,208420430.0,264000000.0,206862625.0,0.007531,0.789471,No
2,mgl,C0004,CS1705,OUTLET PM REST AREA KM 72A PURWAKARTA,SSSG,Derryanto,None,June,299813825.0,276000000.0,225053900.0,0.332187,1.086282,Yes
3,mgl,C0005,CS1710,OUTLET SZ REST AREA KM 72A PURWAKARTA,SSSG,Derryanto,None,June,247472908.0,371000000.0,285395530.0,-0.132877,0.667043,No
4,mgl,C0006,CS1729,OUTLET SZ REST AREA PENDOPO 456A SEMARANG,SSSG,Isnario,None,June,329512662.0,729000000.0,647940277.0,-0.491446,0.452006,No


## Transformasi absen

In [13]:
df_absen["Company"] = df_absen["Company"].astype("str").fillna("")
df_absen["Company"] = np.where(
    (df_absen["Branch"].str.contains(" SZ "))
    | (df_absen["Branch"].str.contains(" LV "))
    | (df_absen["Branch"].str.contains(" PM ")),
    "mgl",
    "mpr",
)

In [14]:
df_absen = df_absen[
    ~df_absen["Job Position"].isin(["City Coordinator", "SA-Bazaar", "City Head"])
]

df_absen["Jumlah Karyawan"] = df_absen.groupby(["Company", "Branch Code"])[
    "Employee ID"
].transform("count")

df_absen["absenreq"] = np.where(df_absen["Hari Kerja"] >= 25, "Yes", "No")

In [15]:
df_absen.columns = df_absen.columns.str.lower().str.replace(" ", "_")
df_absen = df_absen.rename(columns={"branch_code": "axcode", "company": "dataareaid"})

In [16]:
df_absen["axcode"] = np.where(
    df_absen["axcode"].str.contains("-"),
    df_absen["axcode"].str.split("-").get(0),
    df_absen["axcode"],
)

In [17]:
df_absen.head(5)

,dataareaid,employee_id,full_name,job_position,axcode,branch,hari_kerja,jumlah_karyawan,absenreq
0,mpr,02230085,Rahmat Hidayat,Sales Assistant,C0970,SHOWROOM MZ MALL RATU INDAH II MAKASSAR,24,3,No
1,mpr,02230650,Reni Dwi Andriani,Sales Assistant,C1177,TRANSMART MC JEMBER,26,1,Yes
2,mpr,02230679,Raka Andrian Pratama,Sales Assistant,C1094,SHOWROOM MZ KUNINGAN CITY MALL,26,2,Yes
3,mpr,02230682,Debora Violitha Br Ginting Seragih,Sales Assistant,C0215,MATAHARI MC MEDAN THAMRIN,26,2,Yes
4,mpr,02230684,Indriana Maulidunnisa,Sales Assistant,C0063,SOGO MN CENTRAL PARK,26,1,Yes


## Transformasi Incentive SPG

In [18]:
incentive_spg = df_absen.merge(
    df_sales_master[
        [
            "dataareaid",
            "axcode",
            "stdname",
            "sales_23",
            "sales_24",
            "growth",
            "growthreq",
            "target",
            "achievement",
        ]
    ],
    "left",
    on=["dataareaid", "axcode"],
)

In [19]:
incentive_spg["isincentive"] = np.where(
    (incentive_spg["absenreq"] == "Yes") & (incentive_spg["growthreq"] == "Yes"),
    "Yes",
    "No",
)

# ganti sesuai tahun yang ingin dihitung
incentive_spg["productivity"] = (
    incentive_spg["sales_24"] / incentive_spg["jumlah_karyawan"]
)

In [20]:
def calculate_incentive_sa(df):
    conditions = [
        (df["productivity"] > 80000000) & (df["achievement"] > 1.20),
        (df["productivity"] > 80000000) & (df["achievement"] >= 1),
        (df["productivity"] > 80000000) & (df["achievement"] >= 0.95),
        (df["productivity"] >= 40000000) & (df["achievement"] > 1.20),
        (df["productivity"] >= 40000000) & (df["achievement"] >= 1),
        (df["productivity"] >= 40000000) & (df["achievement"] >= 0.95),
        (df["productivity"] > 0) & (df["achievement"] > 1.20),
        (df["productivity"] > 0) & (df["achievement"] >= 1),
        (df["productivity"] > 0) & (df["achievement"] >= 0.95),
        # (df["Company"] == 'mgl') & (df["productivity"] > 160000000) & (df["achievement"] > 1.20),
        # (df["Company"] == 'mgl') & (df["productivity"] > 160000000) & (df["achievement"] >= 1),
        # (df["Company"] == 'mgl') & (df["productivity"] > 160000000) & (df["achievement"] >= 0.95),
        # (df["Company"] == 'mgl') & (df["productivity"] >= 100000000) & (df["achievement"] > 1.20),
        # (df["Company"] == 'mgl') & (df["productivity"] >= 100000000) & (df["achievement"] >= 1),
        # (df["Company"] == 'mgl') & (df["productivity"] >= 100000000) & (df["achievement"] >= 0.95),
        # (df["Company"] == 'mgl') & (df["productivity"] > 0) & (df["achievement"] > 1.20),
        # (df["Company"] == 'mgl') & (df["productivity"] > 0) & (df["achievement"] >= 1),
        # (df["Company"] == 'mgl') & (df["productivity"] > 0) & (df["achievement"] >= 0.95),
    ]

    outputs = [
        600000,
        400000,
        250000,
        450000,
        300000,
        200000,
        300000,
        200000,
        150000,
        # 600000, 400000, 250000,
        # 450000, 300000, 200000,
        # 300000, 200000, 150000
    ]

    df["calculated_incentive"] = np.select(conditions, outputs, default=0)

    return df

In [21]:
def calculate_incentive_sh_ch(df):
    conditions = [
        (df["achievement"] > 1.20),
        (df["achievement"] >= 1),
        (df["achievement"] >= 0.95),
    ]
    outputs = [750000, 500000, 350000]
    df["calculated_incentive"] = np.select(conditions, outputs, default=0)
    return df

In [22]:
def calculated_incentive(df):
    spg = df[
        df["job_position"].isin(
            [
                "Sales Assistant",
                "Sa-Bazaar (Reguler)",
            ]
        )
    ]
    other = df[
        ~df["job_position"].isin(
            [
                "Sales Assistant",
                "Sa-Bazaar (Reguler)",
            ]
        )
    ]

    other = calculate_incentive_sh_ch(other)
    spg = calculate_incentive_sa(spg)
    df = pd.concat([spg, other])
    return df


# Menghitung jumlah incentive yang akan didapatkan
# incentive_spg["calculated_incentive"] = incentive_spg.apply(calculate_incentive, axis=1)
incentive_spg = calculated_incentive(incentive_spg)

In [23]:
# Menghitung final incentive yang akan didapatkan oleh spg
incentive_spg["final_incentive"] = np.where(
    (incentive_spg["isincentive"] == "Yes"),  # Syarat spg dan toko yang dapat incentive
    incentive_spg["calculated_incentive"],
    0,
)

## Transform Area Head Senior

In [24]:
df_am = pd.read_excel("Copy of DATA AM UNTUK NOVAL IT(1).xlsx")

In [25]:
df_am.loc[df_am["Full Name"] == "Derriyanto", "Full Name"] = "Derryanto"
df_am.loc[df_am["Full Name"] == "Erwin Syah", "Full Name"] = "Erwinsyah"
df_am.loc[df_am["Full Name"] == "Edi Sonne", "Full Name"] = "Edisone"
df_am.loc[df_am["Full Name"] == "Faridha", "Full Name"] = "Farida"
df_am.loc[df_am["Full Name"] == "Dewi Maya Sari", "Full Name"] = "Dewi Maya Sari Ss"
df_am.loc[df_am["Full Name"] == "Priatman Syukur Telaum Banua", "Full Name"] = (
    "Priatman Syukur Telaumbanua"
)
df_am.loc[df_am["Full Name"] == "Ratu Adinda Meydiatari PN", "Full Name"] = (
    "Ratu Adinda Meidiatari Permata Noor"
)
df_am.loc[df_am["Full Name"] == "Mita Noviyana Kusumaningtyas", "Full Name"] = (
    "Mita Noviana"
)

In [26]:
df_sales_master = (
    df_sales_master.merge(
        df_am[["Full Name", "Job Position"]].rename(
            columns={"Full Name": "areahead", "Job Position": "job_position"}
        ),
        "left",
        on="areahead",
    )
)

In [27]:
df_sales_master["job_position"] = df_sales_master["job_position"].fillna("Area Manager")

In [28]:
incentive_areahead = (
    df_sales_master.query("job_position == 'Area Manager'")
    .groupby(["areahead", "month"], as_index=False)
    .agg(
        {
            "target": "sum",
            "sales_23": "sum",
            "sales_24": "sum",
        }
    )
    .merge(
        df_sales_master.query("job_position == 'Area Manager' & sssg == 'SSSG'")
        .groupby(["areahead", "month"], as_index=False)
        .agg({"sales_23": "sum", "sales_24": "sum"}),
        "left",
        on=["areahead", "month"],
        suffixes=("", "_sssg"),
    )
    .fillna(0)
)

incentive_areahead["growth"] = (
    incentive_areahead["sales_24_sssg"] - incentive_areahead["sales_23_sssg"]
) / incentive_areahead["sales_23_sssg"]
incentive_areahead["growth"] = incentive_areahead["growth"].replace(
    [np.inf, -np.inf, np.nan], 0
)

incentive_areahead = calculate_growth_req(incentive_areahead)

incentive_areahead = calculate_ach(incentive_areahead)

In [29]:
conditions = [
    incentive_areahead["achievement"] > 1.20,
    incentive_areahead["achievement"] >= 1,
    incentive_areahead["achievement"] >= 0.95,
]

results = [8000000, 5000000, 3000000]

incentive_areahead["incentive"] = np.select(conditions, results, default=0)
incentive_areahead["final_incentive"] = np.where(
    incentive_areahead["growthreq"] == "Yes", incentive_areahead["incentive"], 0
)

## Transformasi Area Head Junior

In [30]:
incentive_areahead_junior = (
    df_sales_master.query("job_position == 'Junior Area Manager'")
    .groupby(["areahead", "month"], as_index=False)
    .agg(
        {
            "target": "sum",
            "sales_23": "sum",
            "sales_24": "sum",
        }
    )
    .merge(
        df_sales_master.query("job_position == 'Junior Area Manager' & sssg == 'SSSG'")
        .groupby(["areahead", "month"], as_index=False)
        .agg({"sales_23": "sum", "sales_24": "sum"}),
        "left",
        on=["areahead", "month"],
        suffixes=("", "_sssg"),
    )
    .fillna(0)
)

incentive_areahead_junior["growth"] = (
    incentive_areahead_junior["sales_24_sssg"] - incentive_areahead_junior["sales_23_sssg"]
) / incentive_areahead_junior["sales_23_sssg"]
incentive_areahead_junior["growth"] = incentive_areahead_junior["growth"].replace(
    [np.inf, -np.inf, np.nan], 0
)

incentive_areahead_junior = calculate_growth_req(incentive_areahead_junior)

incentive_areahead_junior = calculate_ach(incentive_areahead_junior)

In [31]:
conditions = [
    incentive_areahead_junior["achievement"] > 1.20,
    incentive_areahead_junior["achievement"] >= 1,
    incentive_areahead_junior["achievement"] >= 0.95,
]

results = [5000000, 3000000, 2000000]

incentive_areahead_junior["incentive"] = np.select(conditions, results, default=0)
incentive_areahead_junior["final_incentive"] = np.where(
    incentive_areahead_junior["growthreq"] == "Yes", incentive_areahead_junior["incentive"], 0
)

## Transformasi City Head

In [32]:
incentive_cityhead = (
    df_sales_master.groupby(["cityhead", "month"], as_index=False)
    .agg(
        {
            "target": "sum",
            "sales_23": "sum",
            "sales_24": "sum",
        }
    )
    .merge(
        df_sales_master.query("sssg == 'SSSG'")
        .groupby(["cityhead", "month"], as_index=False)
        .agg({"sales_23": "sum", "sales_24": "sum"}),
        "left",
        on=["cityhead", "month"],
        suffixes=("", "_sssg"),
    )
    .fillna(0)
)

incentive_cityhead["growth"] = (
    incentive_cityhead["sales_24_sssg"] - incentive_cityhead["sales_23_sssg"]
) / incentive_cityhead["sales_23_sssg"]
incentive_cityhead["growth"] = incentive_cityhead["growth"].replace(
    [np.inf, -np.inf, np.nan], 0
)

incentive_cityhead = calculate_growth_req(incentive_cityhead)

incentive_cityhead = calculate_ach(incentive_cityhead)

In [33]:
conditions = [
    incentive_cityhead["achievement"] > 1.20,
    incentive_cityhead["achievement"] >= 1,
    incentive_cityhead["achievement"] >= 0.95,
]

results = [2500000, 1500000, 1000000]

incentive_cityhead["incentive"] = np.select(conditions, results, default=0)
incentive_cityhead["final_incentive"] = np.where(
    incentive_cityhead["growthreq"] == "Yes", incentive_cityhead["incentive"], 0
)

# Load

In [34]:
incentive_spg.columns = incentive_spg.columns.str.replace("_", " ").str.title()
incentive_areahead.columns = incentive_areahead.columns.str.replace(
    "_", " "
).str.title()
incentive_areahead_junior.columns = incentive_areahead_junior.columns.str.replace(
    "_", " "
).str.title()
incentive_cityhead.columns = incentive_cityhead.columns.str.replace(
    "_", " "
).str.title()
df_sales_master.columns = df_sales_master.columns.str.replace("_", " ").str.title()

In [35]:
with pd.ExcelWriter(f"Data Insentive Juni 2024.xlsx") as wr:
    df_sales_master.to_excel(wr, sheet_name="Sales", index=False)

    incentive_spg.to_excel(wr, sheet_name="Sales Assistant All", index=False)

    incentive_spg[(incentive_spg["Dataareaid"] == "mpr")].to_excel(
        wr, sheet_name="Sales Assistant MP", index=False
    )

    incentive_spg[(incentive_spg["Dataareaid"] == "mgl")].to_excel(
        wr, sheet_name="Sales Assistant MG", index=False
    )

    incentive_areahead.to_excel(wr, sheet_name="Area Head Senior", index=False)
    incentive_areahead_junior.to_excel(wr, sheet_name="Area Head Junior", index=False)

    df_sales_master.drop(columns="Cityhead").to_excel(
        wr, sheet_name="Area Head Detail", index=False
    )

    incentive_cityhead.to_excel(wr, sheet_name="City Head", index=False)

    df_sales_master.drop(columns="Areahead").to_excel(
        wr, sheet_name="City Head Detail", index=False
    )